In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import datetime
from sklearn.model_selection import train_test_split
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display

#pandas：
pd.set_option('display.max_rows', 5000)
pd.set_option('display.max_columns', 5000)
pd.set_option('display.width', 1000)

train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')

In [2]:
# Concatenating train & test
test_id = test['id']
train_sales = train['sales']

train['istrain'] = 1
test['istrain'] = 0
df = pd.concat([train,test.drop(['id'],1)], sort=False)
df['date'] = pd.to_datetime(df['date'])
print('Combined df shape:{}'.format(df.shape))

Combined df shape:(958000, 5)


## 1. Feature Engineering

#### 1.1 time features

In [3]:
# Extracting date features
df['dayofmonth'] = df.date.dt.day
df['dayofyear'] = df.date.dt.dayofyear
df['dayofweek'] = df.date.dt.dayofweek
df['weekofyear'] = df.date.dt.week
df['month'] = df.date.dt.month
df['quarter'] = df.date.dt.quarter
df['year'] = df.date.dt.year
df['weekofyear'] = df.date.dt.weekofyear
df['is_month_start'] = (df.date.dt.is_month_start).astype(int)
df['is_month_end'] = (df.date.dt.is_month_end).astype(int)
df.head(2)

,date,store,item,sales,istrain,dayofmonth,dayofyear,dayofweek,weekofyear,month,quarter,year,is_month_start,is_month_end
0,2013-01-01,1,1,13.0,1,1,1,1,1,1,1,2013,1,0
1,2013-01-02,1,1,11.0,1,2,2,2,1,1,1,2013,0,0


In [4]:
df.shape

(958000, 14)

In [5]:
#df.sort_values(by=['store','item','date'], axis=0, inplace=True)

#### 1.2 aggregated sales values

In [6]:
def create_sales_agg_features(df, gpby_cols, target_col, agg_funcs):
    '''
    Creates various sales agg features with given agg functions  
    '''
    gpby = df.groupby(gpby_cols)
    newdf = df[gpby_cols].drop_duplicates().reset_index(drop=True)
    for agg_name, agg_func in agg_funcs.items():
        aggdf = gpby[target_col].agg(agg_func).reset_index()
        aggdf.rename(columns={target_col:target_col+'_'+agg_name}, inplace=True)
        newdf = newdf.merge(aggdf, on=gpby_cols, how='left')
    for c in newdf.columns:
        if c not in gpby_cols:
            newdf = newdf.rename(columns={c: gpby_cols[-1]+'_'+c})
    return newdf

In [7]:
# Creating sales dayofmonth aggregated values
dayofmonth_agg = create_sales_agg_features(df,gpby_cols=['store','item','dayofmonth'], 
                                        target_col='sales', 
                                        agg_funcs={'mean':np.mean, 
                                             'median':np.median, 'max':np.max, 
                                             'min':np.min, 'std':np.std})
# Creating sales monthwise aggregated values
dayofyear_agg = create_sales_agg_features(df,gpby_cols=['store','item','dayofyear'], 
                                        target_col='sales', 
                                        agg_funcs={'mean':np.mean, 
                                             'median':np.median, 'max':np.max, 
                                             'min':np.min, 'std':np.std})
# Creating sales monthwise aggregated values
dayofweek_agg = create_sales_agg_features(df,gpby_cols=['store','item','dayofweek'], 
                                        target_col='sales', 
                                        agg_funcs={'mean':np.mean, 
                                             'median':np.median, 'max':np.max, 
                                             'min':np.min, 'std':np.std})
# Creating sales monthwise aggregated values
weekofyear_agg = create_sales_agg_features(df,gpby_cols=['store','item','weekofyear'], 
                                        target_col='sales', 
                                        agg_funcs={'mean':np.mean, 
                                             'median':np.median, 'max':np.max, 
                                             'min':np.min, 'std':np.std})
# Creating sales monthwise aggregated values
month_agg = create_sales_agg_features(df,gpby_cols=['store','item','month'], 
                                        target_col='sales', 
                                        agg_funcs={'mean':np.mean, 
                                             'median':np.median, 'max':np.max, 
                                             'min':np.min, 'std':np.std})
# Creating sales quarterwise aggregated values
quarter_agg = create_sales_agg_features(df,gpby_cols=['store','item','quarter'], 
                                        target_col='sales', 
                                        agg_funcs={'mean':np.mean, 
                                             'median':np.median, 'max':np.max, 
                                             'min':np.min, 'std':np.std})
# Creating sales yearwise aggregated values
year_agg = create_sales_agg_features(df,gpby_cols=['store','item','year'], 
                                        target_col='sales', 
                                        agg_funcs={'mean':np.mean, 
                                             'median':np.median, 'max':np.max, 
                                             'min':np.min, 'std':np.std})

In [8]:
df = df.merge(dayofmonth_agg,on = ['store','item','dayofmonth'])
df = df.merge(dayofyear_agg,on = ['store','item','dayofyear'])
df = df.merge(dayofweek_agg,on = ['store','item','dayofweek'])
df = df.merge(weekofyear_agg,on = ['store','item','weekofyear'])
df = df.merge(month_agg,on = ['store','item','month'])
df = df.merge(quarter_agg,on = ['store','item','quarter'])
df = df.merge(year_agg,on = ['store','item','year'])

In [9]:
df.head(2)

,date,store,item,sales,istrain,dayofmonth,dayofyear,dayofweek,weekofyear,month,quarter,year,is_month_start,is_month_end,dayofmonth_sales_mean,dayofmonth_sales_median,dayofmonth_sales_max,dayofmonth_sales_min,dayofmonth_sales_std,dayofyear_sales_mean,dayofyear_sales_median,dayofyear_sales_max,dayofyear_sales_min,dayofyear_sales_std,dayofweek_sales_mean,dayofweek_sales_median,dayofweek_sales_max,dayofweek_sales_min,dayofweek_sales_std,weekofyear_sales_mean,weekofyear_sales_median,weekofyear_sales_max,weekofyear_sales_min,weekofyear_sales_std,month_sales_mean,month_sales_median,month_sales_max,month_sales_min,month_sales_std,quarter_sales_mean,quarter_sales_median,quarter_sales_max,quarter_sales_min,quarter_sales_std,year_sales_mean,year_sales_median,year_sales_max,year_sales_min,year_sales_std
0,2013-01-01,1,1,13.0,1,1,1,1,1,1,1,2013,1,0,19.916667,19.0,42.0,9.0,6.785557,14.6,13.0,21.0,9.0,5.176872,18.168582,18.0,32.0,5.0,5.819259,13.970588,13.5,26.0,5.0,4.238963,13.709677,13.0,29.0,5.0,4.397413,15.334812,15.0,31.0,4.0,4.826879,16.506849,16.0,38.0,5.0,5.568069
1,2013-01-02,1,1,11.0,1,2,2,2,1,1,1,2013,0,0,18.716667,19.0,38.0,6.0,6.036869,14.4,14.0,19.0,11.0,2.966479,18.793103,19.0,50.0,6.0,5.943590,13.970588,13.5,26.0,5.0,4.238963,13.709677,13.0,29.0,5.0,4.397413,15.334812,15.0,31.0,4.0,4.826879,16.506849,16.0,38.0,5.0,5.568069


#### 1.3 Features constructed from previous sales values

In [10]:
# Creating sales lag features
def create_sales_lag_feats(df, gpby_cols, target_col, lags):
    gpby = df.groupby(gpby_cols)
    for i in lags:
        df['_'.join([target_col, 'lag', str(i)])] = \
                gpby[target_col].shift(i).values + np.random.normal(scale=1.6, size=(len(df),))
    return df

# Creating sales rolling mean features
def create_sales_rmean_feats(df, gpby_cols, target_col, windows, min_periods=2, 
                             shift=1, win_type=None):
    gpby = df.groupby(gpby_cols)
    for w in windows:
        df['_'.join([target_col, 'rmean', str(w)])] = \
            gpby[target_col].shift(shift).rolling(window=w, 
                                                  min_periods=min_periods,
                                                  win_type=win_type).mean().values +\
            np.random.normal(scale=1.6, size=(len(df),))
    return df

# Creating sales rolling median features
def create_sales_rmed_feats(df, gpby_cols, target_col, windows, min_periods=2, 
                            shift=1, win_type=None):
    gpby = df.groupby(gpby_cols)
    for w in windows:
        df['_'.join([target_col, 'rmed', str(w)])] = \
            gpby[target_col].shift(shift).rolling(window=w, 
                                                  min_periods=min_periods,
                                                  win_type=win_type).median().values +\
            np.random.normal(scale=1.6, size=(len(df),))
    return df

# Creating sales exponentially weighted mean features
def create_sales_ewm_feats(df, gpby_cols, target_col, alpha=[0.9], shift=[1]):
    gpby = df.groupby(gpby_cols)
    for a in alpha:
        for s in shift:
            df['_'.join([target_col, 'lag', str(s), 'ewm', str(a)])] = \
                gpby[target_col].shift(s).ewm(alpha=a).mean().values
    return df

In [11]:
# Creating sales lag, rolling mean, rolling median, ohe features of the above train set
df_whole = create_sales_lag_feats(df, gpby_cols=['store','item'], target_col='sales', 
                                  lags=[91,98,105,112,119,126,182,364,546,728])
df_whole = create_sales_rmean_feats(df_whole, gpby_cols=['store','item'], 
                                    target_col='sales', windows=[364,546], 
                                    min_periods=10, win_type='triang')
# df = create_sales_rmed_feats(df, gpby_cols=['store','item'], target_col='sales', 
#                              windows=[364,546], min_periods=2) #98,119,
df_whole = create_sales_ewm_feats(df_whole, gpby_cols=['store','item'], target_col='sales', 
                                  alpha=[0.95, 0.9, 0.8, 0.7, 0.6, 0.5], 
                                  shift=[91,98,105,112,119,126,182,364,546,728])


In [12]:
#'dayofmonth',,'weekofyear'
df_whole = df_whole.fillna(method='bfill')
df_whole = df_whole.fillna(method='ffill')

#### 1.4 One Hot Encoding

In [13]:
def one_hot_encoder(df, ohe_cols=['store','item']):
    '''
    One-Hot Encoder function
    '''
    print('Creating OHE features..\nOld df shape:{}'.format(df.shape))
    df = pd.get_dummies(df, columns=ohe_cols)
    print('New df shape:{}'.format(df.shape))
    return df

In [14]:
# One-Hot Encoding
df_whole = one_hot_encoder(df_whole, ohe_cols=['store','item','month','quarter','year'])


Creating OHE features..
Old df shape:(958000, 121)
New df shape:(958000, 198)


In [15]:
df_whole.head(2)

,date,sales,istrain,dayofmonth,dayofyear,dayofweek,weekofyear,is_month_start,is_month_end,dayofmonth_sales_mean,dayofmonth_sales_median,dayofmonth_sales_max,dayofmonth_sales_min,dayofmonth_sales_std,dayofyear_sales_mean,dayofyear_sales_median,dayofyear_sales_max,dayofyear_sales_min,dayofyear_sales_std,dayofweek_sales_mean,dayofweek_sales_median,dayofweek_sales_max,dayofweek_sales_min,dayofweek_sales_std,weekofyear_sales_mean,weekofyear_sales_median,weekofyear_sales_max,weekofyear_sales_min,weekofyear_sales_std,month_sales_mean,month_sales_median,month_sales_max,month_sales_min,month_sales_std,quarter_sales_mean,quarter_sales_median,quarter_sales_max,quarter_sales_min,quarter_sales_std,year_sales_mean,year_sales_median,year_sales_max,year_sales_min,year_sales_std,sales_lag_91,sales_lag_98,sales_lag_105,sales_lag_112,sales_lag_119,sales_lag_126,sales_lag_182,sales_lag_364,sales_lag_546,sales_lag_728,sales_rmean_364,sales_rmean_546,sales_lag_91_ewm_0.95,sales_lag_98_ewm_0.95,sales_lag_105_ewm_0.95,sales_lag_112_ewm_0.95,sales_lag_119_ewm_0.95,sales_lag_126_ewm_0.95,sales_lag_182_ewm_0.95,sales_lag_364_ewm_0.95,sales_lag_546_ewm_0.95,sales_lag_728_ewm_0.95,sales_lag_91_ewm_0.9,sales_lag_98_ewm_0.9,sales_lag_105_ewm_0.9,sales_lag_112_ewm_0.9,sales_lag_119_ewm_0.9,sales_lag_126_ewm_0.9,sales_lag_182_ewm_0.9,sales_lag_364_ewm_0.9,sales_lag_546_ewm_0.9,sales_lag_728_ewm_0.9,sales_lag_91_ewm_0.8,sales_lag_98_ewm_0.8,sales_lag_105_ewm_0.8,sales_lag_112_ewm_0.8,sales_lag_119_ewm_0.8,sales_lag_126_ewm_0.8,sales_lag_182_ewm_0.8,sales_lag_364_ewm_0.8,sales_lag_546_ewm_0.8,sales_lag_728_ewm_0.8,sales_lag_91_ewm_0.7,sales_lag_98_ewm_0.7,sales_lag_105_ewm_0.7,sales_lag_112_ewm_0.7,sales_lag_119_ewm_0.7,sales_lag_126_ewm_0.7,sales_lag_182_ewm_0.7,sales_lag_364_ewm_0.7,sales_lag_546_ewm_0.7,sales_lag_728_ewm_0.7,sales_lag_91_ewm_0.6,sales_lag_98_ewm_0.6,sales_lag_105_ewm_0.6,sales_lag_112_ewm_0.6,sales_lag_119_ewm_0.6,sales_lag_126_ewm_0.6,sales_lag_182_ewm_0.6,sales_lag_364_ewm_0.6,sales_lag_546_ewm_0.6,sales_lag_728_ewm_0.6,sales_lag_91_ewm_0.5,sales_lag_98_ewm_0.5,sales_lag_105_ewm_0.5,sales_lag_112_ewm_0.5,sales_lag_119_ewm_0.5,sales_lag_126_ewm_0.5,sales_lag_182_ewm_0.5,sales_lag_364_ewm_0.5,sales_lag_546_ewm_0.5,sales_lag_728_ewm_0.5,store_1,store_2,store_3,store_4,store_5,store_6,store_7,store_8,store_9,store_10,item_1,item_2,item_3,item_4,item_5,item_6,item_7,item_8,item_9,item_10,item_11,item_12,item_13,item_14,item_15,item_16,item_17,item_18,item_19,item_20,item_21,item_22,item_23,item_24,item_25,item_26,item_27,item_28,item_29,item_30,item_31,item_32,item_33,item_34,item_35,item_36,item_37,item_38,item_39,item_40,item_41,item_42,item_43,item_44,item_45,item_46,item_47,item_48,item_49,item_50,month_1,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12,quarter_1,quarter_2,quarter_3,quarter_4,year_2013,year_2014,year_2015,year_2016,year_2017,year_2018
0,2013-01-01,13.0,1,1,1,1,1,1,0,19.916667,19.0,42.0,9.0,6.785557,14.6,13.0,21.0,9.0,5.176872,18.168582,18.0,32.0,5.0,5.819259,13.970588,13.5,26.0,5.0,4.238963,13.709677,13.0,29.0,5.0,4.397413,15.334812,15.0,31.0,4.0,4.826879,16.506849,16.0,38.0,5.0,5.568069,12.371936,14.409526,15.092037,14.605592,12.826156,16.049444,12.772816,13.687095,12.05477,12.75611,10.928626,11.068115,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0
1,2013-01-02,11.0,1,2,2,2,1,0,0,18.716667,19.0,38.0,6.0,6.036869,14.4,14.0,19.0,11.0,2.966479,18.793103,19.0,50.0,6.0,5.943590,13.970588,13.5,26.0,5.0,4.238963,13.709677,13.0,29.0,5.0,4.397413,15.334812,15.0,31.0,4.0,4.826879,16

In [16]:
df_sales = df_whole['sales']
df_istrain = df_whole['istrain']
df_traintest = df_whole.drop(['sales','date','istrain'],1)
df_traintest.shape

(958000, 195)

In [17]:
df_traintest.head(2)

,dayofmonth,dayofyear,dayofweek,weekofyear,is_month_start,is_month_end,dayofmonth_sales_mean,dayofmonth_sales_median,dayofmonth_sales_max,dayofmonth_sales_min,dayofmonth_sales_std,dayofyear_sales_mean,dayofyear_sales_median,dayofyear_sales_max,dayofyear_sales_min,dayofyear_sales_std,dayofweek_sales_mean,dayofweek_sales_median,dayofweek_sales_max,dayofweek_sales_min,dayofweek_sales_std,weekofyear_sales_mean,weekofyear_sales_median,weekofyear_sales_max,weekofyear_sales_min,weekofyear_sales_std,month_sales_mean,month_sales_median,month_sales_max,month_sales_min,month_sales_std,quarter_sales_mean,quarter_sales_median,quarter_sales_max,quarter_sales_min,quarter_sales_std,year_sales_mean,year_sales_median,year_sales_max,year_sales_min,year_sales_std,sales_lag_91,sales_lag_98,sales_lag_105,sales_lag_112,sales_lag_119,sales_lag_126,sales_lag_182,sales_lag_364,sales_lag_546,sales_lag_728,sales_rmean_364,sales_rmean_546,sales_lag_91_ewm_0.95,sales_lag_98_ewm_0.95,sales_lag_105_ewm_0.95,sales_lag_112_ewm_0.95,sales_lag_119_ewm_0.95,sales_lag_126_ewm_0.95,sales_lag_182_ewm_0.95,sales_lag_364_ewm_0.95,sales_lag_546_ewm_0.95,sales_lag_728_ewm_0.95,sales_lag_91_ewm_0.9,sales_lag_98_ewm_0.9,sales_lag_105_ewm_0.9,sales_lag_112_ewm_0.9,sales_lag_119_ewm_0.9,sales_lag_126_ewm_0.9,sales_lag_182_ewm_0.9,sales_lag_364_ewm_0.9,sales_lag_546_ewm_0.9,sales_lag_728_ewm_0.9,sales_lag_91_ewm_0.8,sales_lag_98_ewm_0.8,sales_lag_105_ewm_0.8,sales_lag_112_ewm_0.8,sales_lag_119_ewm_0.8,sales_lag_126_ewm_0.8,sales_lag_182_ewm_0.8,sales_lag_364_ewm_0.8,sales_lag_546_ewm_0.8,sales_lag_728_ewm_0.8,sales_lag_91_ewm_0.7,sales_lag_98_ewm_0.7,sales_lag_105_ewm_0.7,sales_lag_112_ewm_0.7,sales_lag_119_ewm_0.7,sales_lag_126_ewm_0.7,sales_lag_182_ewm_0.7,sales_lag_364_ewm_0.7,sales_lag_546_ewm_0.7,sales_lag_728_ewm_0.7,sales_lag_91_ewm_0.6,sales_lag_98_ewm_0.6,sales_lag_105_ewm_0.6,sales_lag_112_ewm_0.6,sales_lag_119_ewm_0.6,sales_lag_126_ewm_0.6,sales_lag_182_ewm_0.6,sales_lag_364_ewm_0.6,sales_lag_546_ewm_0.6,sales_lag_728_ewm_0.6,sales_lag_91_ewm_0.5,sales_lag_98_ewm_0.5,sales_lag_105_ewm_0.5,sales_lag_112_ewm_0.5,sales_lag_119_ewm_0.5,sales_lag_126_ewm_0.5,sales_lag_182_ewm_0.5,sales_lag_364_ewm_0.5,sales_lag_546_ewm_0.5,sales_lag_728_ewm_0.5,store_1,store_2,store_3,store_4,store_5,store_6,store_7,store_8,store_9,store_10,item_1,item_2,item_3,item_4,item_5,item_6,item_7,item_8,item_9,item_10,item_11,item_12,item_13,item_14,item_15,item_16,item_17,item_18,item_19,item_20,item_21,item_22,item_23,item_24,item_25,item_26,item_27,item_28,item_29,item_30,item_31,item_32,item_33,item_34,item_35,item_36,item_37,item_38,item_39,item_40,item_41,item_42,item_43,item_44,item_45,item_46,item_47,item_48,item_49,item_50,month_1,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12,quarter_1,quarter_2,quarter_3,quarter_4,year_2013,year_2014,year_2015,year_2016,year_2017,year_2018
0,1,1,1,1,1,0,19.916667,19.0,42.0,9.0,6.785557,14.6,13.0,21.0,9.0,5.176872,18.168582,18.0,32.0,5.0,5.819259,13.970588,13.5,26.0,5.0,4.238963,13.709677,13.0,29.0,5.0,4.397413,15.334812,15.0,31.0,4.0,4.826879,16.506849,16.0,38.0,5.0,5.568069,12.371936,14.409526,15.092037,14.605592,12.826156,16.049444,12.772816,13.687095,12.05477,12.75611,10.928626,11.068115,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0
1,2,2,2,1,0,0,18.716667,19.0,38.0,6.0,6.036869,14.4,14.0,19.0,11.0,2.966479,18.793103,19.0,50.0,6.0,5.943590,13.970588,13.5,26.0,5.0,4.238963,13.709677,13.0,29.0,5.0,4.397413,15.334812,15.0,31.0,4.0,4.826879,16.506849,16.0,38.0,5.0,5.568069,12.371936,14.409526,15.0

## 2. Train Test Split
sample data from each store_item

In [18]:
df_sample = pd.concat([df[['store','item','istrain','sales']],df_traintest],1)
df_train_sample = df_sample[
    df_sample.istrain == 1
]

print(df_train_sample.shape)
df_train_sample.head(2)

(913000, 199)


,store,item,istrain,sales,dayofmonth,dayofyear,dayofweek,weekofyear,is_month_start,is_month_end,dayofmonth_sales_mean,dayofmonth_sales_median,dayofmonth_sales_max,dayofmonth_sales_min,dayofmonth_sales_std,dayofyear_sales_mean,dayofyear_sales_median,dayofyear_sales_max,dayofyear_sales_min,dayofyear_sales_std,dayofweek_sales_mean,dayofweek_sales_median,dayofweek_sales_max,dayofweek_sales_min,dayofweek_sales_std,weekofyear_sales_mean,weekofyear_sales_median,weekofyear_sales_max,weekofyear_sales_min,weekofyear_sales_std,month_sales_mean,month_sales_median,month_sales_max,month_sales_min,month_sales_std,quarter_sales_mean,quarter_sales_median,quarter_sales_max,quarter_sales_min,quarter_sales_std,year_sales_mean,year_sales_median,year_sales_max,year_sales_min,year_sales_std,sales_lag_91,sales_lag_98,sales_lag_105,sales_lag_112,sales_lag_119,sales_lag_126,sales_lag_182,sales_lag_364,sales_lag_546,sales_lag_728,sales_rmean_364,sales_rmean_546,sales_lag_91_ewm_0.95,sales_lag_98_ewm_0.95,sales_lag_105_ewm_0.95,sales_lag_112_ewm_0.95,sales_lag_119_ewm_0.95,sales_lag_126_ewm_0.95,sales_lag_182_ewm_0.95,sales_lag_364_ewm_0.95,sales_lag_546_ewm_0.95,sales_lag_728_ewm_0.95,sales_lag_91_ewm_0.9,sales_lag_98_ewm_0.9,sales_lag_105_ewm_0.9,sales_lag_112_ewm_0.9,sales_lag_119_ewm_0.9,sales_lag_126_ewm_0.9,sales_lag_182_ewm_0.9,sales_lag_364_ewm_0.9,sales_lag_546_ewm_0.9,sales_lag_728_ewm_0.9,sales_lag_91_ewm_0.8,sales_lag_98_ewm_0.8,sales_lag_105_ewm_0.8,sales_lag_112_ewm_0.8,sales_lag_119_ewm_0.8,sales_lag_126_ewm_0.8,sales_lag_182_ewm_0.8,sales_lag_364_ewm_0.8,sales_lag_546_ewm_0.8,sales_lag_728_ewm_0.8,sales_lag_91_ewm_0.7,sales_lag_98_ewm_0.7,sales_lag_105_ewm_0.7,sales_lag_112_ewm_0.7,sales_lag_119_ewm_0.7,sales_lag_126_ewm_0.7,sales_lag_182_ewm_0.7,sales_lag_364_ewm_0.7,sales_lag_546_ewm_0.7,sales_lag_728_ewm_0.7,sales_lag_91_ewm_0.6,sales_lag_98_ewm_0.6,sales_lag_105_ewm_0.6,sales_lag_112_ewm_0.6,sales_lag_119_ewm_0.6,sales_lag_126_ewm_0.6,sales_lag_182_ewm_0.6,sales_lag_364_ewm_0.6,sales_lag_546_ewm_0.6,sales_lag_728_ewm_0.6,sales_lag_91_ewm_0.5,sales_lag_98_ewm_0.5,sales_lag_105_ewm_0.5,sales_lag_112_ewm_0.5,sales_lag_119_ewm_0.5,sales_lag_126_ewm_0.5,sales_lag_182_ewm_0.5,sales_lag_364_ewm_0.5,sales_lag_546_ewm_0.5,sales_lag_728_ewm_0.5,store_1,store_2,store_3,store_4,store_5,store_6,store_7,store_8,store_9,store_10,item_1,item_2,item_3,item_4,item_5,item_6,item_7,item_8,item_9,item_10,item_11,item_12,item_13,item_14,item_15,item_16,item_17,item_18,item_19,item_20,item_21,item_22,item_23,item_24,item_25,item_26,item_27,item_28,item_29,item_30,item_31,item_32,item_33,item_34,item_35,item_36,item_37,item_38,item_39,item_40,item_41,item_42,item_43,item_44,item_45,item_46,item_47,item_48,item_49,item_50,month_1,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12,quarter_1,quarter_2,quarter_3,quarter_4,year_2013,year_2014,year_2015,year_2016,year_2017,year_2018
0,1,1,1,13.0,1,1,1,1,1,0,19.916667,19.0,42.0,9.0,6.785557,14.6,13.0,21.0,9.0,5.176872,18.168582,18.0,32.0,5.0,5.819259,13.970588,13.5,26.0,5.0,4.238963,13.709677,13.0,29.0,5.0,4.397413,15.334812,15.0,31.0,4.0,4.826879,16.506849,16.0,38.0,5.0,5.568069,12.371936,14.409526,15.092037,14.605592,12.826156,16.049444,12.772816,13.687095,12.05477,12.75611,10.928626,11.068115,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0
1,1,1,1,11.0,2,2,2,1,0,0,18.716667,19.0,38.0,6.0,6.036869,14.4,14.0,19.0,11.0,2.966479,18.793103,19.0,50.0,6.0,5.943590,13.970588,13.5,26.0,5.0,4.238963,13.709677,13.0,29.0,5.0,4.397413,15.334812,15.0,31.0,4.0,4.826879,16.506849,

In [19]:
final_test = df_sample[
    df_sample.istrain == 0
].drop(['store','item','istrain','sales'],1)
print(final_test.shape)
final_test.head(2)

(45000, 195)


,dayofmonth,dayofyear,dayofweek,weekofyear,is_month_start,is_month_end,dayofmonth_sales_mean,dayofmonth_sales_median,dayofmonth_sales_max,dayofmonth_sales_min,dayofmonth_sales_std,dayofyear_sales_mean,dayofyear_sales_median,dayofyear_sales_max,dayofyear_sales_min,dayofyear_sales_std,dayofweek_sales_mean,dayofweek_sales_median,dayofweek_sales_max,dayofweek_sales_min,dayofweek_sales_std,weekofyear_sales_mean,weekofyear_sales_median,weekofyear_sales_max,weekofyear_sales_min,weekofyear_sales_std,month_sales_mean,month_sales_median,month_sales_max,month_sales_min,month_sales_std,quarter_sales_mean,quarter_sales_median,quarter_sales_max,quarter_sales_min,quarter_sales_std,year_sales_mean,year_sales_median,year_sales_max,year_sales_min,year_sales_std,sales_lag_91,sales_lag_98,sales_lag_105,sales_lag_112,sales_lag_119,sales_lag_126,sales_lag_182,sales_lag_364,sales_lag_546,sales_lag_728,sales_rmean_364,sales_rmean_546,sales_lag_91_ewm_0.95,sales_lag_98_ewm_0.95,sales_lag_105_ewm_0.95,sales_lag_112_ewm_0.95,sales_lag_119_ewm_0.95,sales_lag_126_ewm_0.95,sales_lag_182_ewm_0.95,sales_lag_364_ewm_0.95,sales_lag_546_ewm_0.95,sales_lag_728_ewm_0.95,sales_lag_91_ewm_0.9,sales_lag_98_ewm_0.9,sales_lag_105_ewm_0.9,sales_lag_112_ewm_0.9,sales_lag_119_ewm_0.9,sales_lag_126_ewm_0.9,sales_lag_182_ewm_0.9,sales_lag_364_ewm_0.9,sales_lag_546_ewm_0.9,sales_lag_728_ewm_0.9,sales_lag_91_ewm_0.8,sales_lag_98_ewm_0.8,sales_lag_105_ewm_0.8,sales_lag_112_ewm_0.8,sales_lag_119_ewm_0.8,sales_lag_126_ewm_0.8,sales_lag_182_ewm_0.8,sales_lag_364_ewm_0.8,sales_lag_546_ewm_0.8,sales_lag_728_ewm_0.8,sales_lag_91_ewm_0.7,sales_lag_98_ewm_0.7,sales_lag_105_ewm_0.7,sales_lag_112_ewm_0.7,sales_lag_119_ewm_0.7,sales_lag_126_ewm_0.7,sales_lag_182_ewm_0.7,sales_lag_364_ewm_0.7,sales_lag_546_ewm_0.7,sales_lag_728_ewm_0.7,sales_lag_91_ewm_0.6,sales_lag_98_ewm_0.6,sales_lag_105_ewm_0.6,sales_lag_112_ewm_0.6,sales_lag_119_ewm_0.6,sales_lag_126_ewm_0.6,sales_lag_182_ewm_0.6,sales_lag_364_ewm_0.6,sales_lag_546_ewm_0.6,sales_lag_728_ewm_0.6,sales_lag_91_ewm_0.5,sales_lag_98_ewm_0.5,sales_lag_105_ewm_0.5,sales_lag_112_ewm_0.5,sales_lag_119_ewm_0.5,sales_lag_126_ewm_0.5,sales_lag_182_ewm_0.5,sales_lag_364_ewm_0.5,sales_lag_546_ewm_0.5,sales_lag_728_ewm_0.5,store_1,store_2,store_3,store_4,store_5,store_6,store_7,store_8,store_9,store_10,item_1,item_2,item_3,item_4,item_5,item_6,item_7,item_8,item_9,item_10,item_11,item_12,item_13,item_14,item_15,item_16,item_17,item_18,item_19,item_20,item_21,item_22,item_23,item_24,item_25,item_26,item_27,item_28,item_29,item_30,item_31,item_32,item_33,item_34,item_35,item_36,item_37,item_38,item_39,item_40,item_41,item_42,item_43,item_44,item_45,item_46,item_47,item_48,item_49,item_50,month_1,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12,quarter_1,quarter_2,quarter_3,quarter_4,year_2013,year_2014,year_2015,year_2016,year_2017,year_2018
365,2,2,1,1,0,0,18.716667,19.0,38.0,6.0,6.036869,14.4,14.0,19.0,11.0,2.966479,18.168582,18.0,32.0,5.0,5.819259,13.970588,13.5,26.0,5.0,4.238963,13.709677,13.0,29.0,5.0,4.397413,15.334812,15.0,31.0,4.0,4.826879,22.183562,22.0,50.0,5.0,6.953278,10.622645,24.179061,17.367162,22.369745,25.022306,22.382114,23.638818,11.235509,12.05477,12.75611,19.036704,16.946334,12.257167,21.003817,18.161768,21.804157,23.577016,23.522597,23.775768,11.095238,13.0,13.0,12.527639,21.011054,18.344289,21.614419,23.206203,23.09050,23.596288,11.181818,13.0,13.0,13.105472,21.016253,18.756419,21.251566,22.610053,22.358679,23.332680,11.333333,13.0,13.0,13.734155,20.989547,19.209125,20.927426,22.187906,21.784448,23.137402,11.461538,13.0,13.0,14.424067,20.923799,19.677926,20.671523,21.912109,21.324462,22.953528,11.571429,13.0,13.0,15.193978,20.829193,20.136764,20.505854,21.749304,20.910864,22.747395,11.666667,13.0,13.0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1
366,3,3,2,1,0,0,20.8666

In [20]:
store_item = []
for a,b in zip(df_train_sample['store'],df_train_sample['item']):
    str_a = '0'+str(a) if a < 10 else str(a)
    str_b = '0'+str(b) if b < 10 else str(b)
    store_item.append(str_a+'_'+str_b)

In [21]:
len(np.unique(store_item))

500

In [22]:
df_train_sample.loc[:,'store_item'] = store_item
df_train_sample = df_train_sample.drop(['store','item','istrain'],1)
df_train_sample.head(2)

/Users/henry/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/Users/henry/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


,sales,dayofmonth,dayofyear,dayofweek,weekofyear,is_month_start,is_month_end,dayofmonth_sales_mean,dayofmonth_sales_median,dayofmonth_sales_max,dayofmonth_sales_min,dayofmonth_sales_std,dayofyear_sales_mean,dayofyear_sales_median,dayofyear_sales_max,dayofyear_sales_min,dayofyear_sales_std,dayofweek_sales_mean,dayofweek_sales_median,dayofweek_sales_max,dayofweek_sales_min,dayofweek_sales_std,weekofyear_sales_mean,weekofyear_sales_median,weekofyear_sales_max,weekofyear_sales_min,weekofyear_sales_std,month_sales_mean,month_sales_median,month_sales_max,month_sales_min,month_sales_std,quarter_sales_mean,quarter_sales_median,quarter_sales_max,quarter_sales_min,quarter_sales_std,year_sales_mean,year_sales_median,year_sales_max,year_sales_min,year_sales_std,sales_lag_91,sales_lag_98,sales_lag_105,sales_lag_112,sales_lag_119,sales_lag_126,sales_lag_182,sales_lag_364,sales_lag_546,sales_lag_728,sales_rmean_364,sales_rmean_546,sales_lag_91_ewm_0.95,sales_lag_98_ewm_0.95,sales_lag_105_ewm_0.95,sales_lag_112_ewm_0.95,sales_lag_119_ewm_0.95,sales_lag_126_ewm_0.95,sales_lag_182_ewm_0.95,sales_lag_364_ewm_0.95,sales_lag_546_ewm_0.95,sales_lag_728_ewm_0.95,sales_lag_91_ewm_0.9,sales_lag_98_ewm_0.9,sales_lag_105_ewm_0.9,sales_lag_112_ewm_0.9,sales_lag_119_ewm_0.9,sales_lag_126_ewm_0.9,sales_lag_182_ewm_0.9,sales_lag_364_ewm_0.9,sales_lag_546_ewm_0.9,sales_lag_728_ewm_0.9,sales_lag_91_ewm_0.8,sales_lag_98_ewm_0.8,sales_lag_105_ewm_0.8,sales_lag_112_ewm_0.8,sales_lag_119_ewm_0.8,sales_lag_126_ewm_0.8,sales_lag_182_ewm_0.8,sales_lag_364_ewm_0.8,sales_lag_546_ewm_0.8,sales_lag_728_ewm_0.8,sales_lag_91_ewm_0.7,sales_lag_98_ewm_0.7,sales_lag_105_ewm_0.7,sales_lag_112_ewm_0.7,sales_lag_119_ewm_0.7,sales_lag_126_ewm_0.7,sales_lag_182_ewm_0.7,sales_lag_364_ewm_0.7,sales_lag_546_ewm_0.7,sales_lag_728_ewm_0.7,sales_lag_91_ewm_0.6,sales_lag_98_ewm_0.6,sales_lag_105_ewm_0.6,sales_lag_112_ewm_0.6,sales_lag_119_ewm_0.6,sales_lag_126_ewm_0.6,sales_lag_182_ewm_0.6,sales_lag_364_ewm_0.6,sales_lag_546_ewm_0.6,sales_lag_728_ewm_0.6,sales_lag_91_ewm_0.5,sales_lag_98_ewm_0.5,sales_lag_105_ewm_0.5,sales_lag_112_ewm_0.5,sales_lag_119_ewm_0.5,sales_lag_126_ewm_0.5,sales_lag_182_ewm_0.5,sales_lag_364_ewm_0.5,sales_lag_546_ewm_0.5,sales_lag_728_ewm_0.5,store_1,store_2,store_3,store_4,store_5,store_6,store_7,store_8,store_9,store_10,item_1,item_2,item_3,item_4,item_5,item_6,item_7,item_8,item_9,item_10,item_11,item_12,item_13,item_14,item_15,item_16,item_17,item_18,item_19,item_20,item_21,item_22,item_23,item_24,item_25,item_26,item_27,item_28,item_29,item_30,item_31,item_32,item_33,item_34,item_35,item_36,item_37,item_38,item_39,item_40,item_41,item_42,item_43,item_44,item_45,item_46,item_47,item_48,item_49,item_50,month_1,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12,quarter_1,quarter_2,quarter_3,quarter_4,year_2013,year_2014,year_2015,year_2016,year_2017,year_2018,store_item
0,13.0,1,1,1,1,1,0,19.916667,19.0,42.0,9.0,6.785557,14.6,13.0,21.0,9.0,5.176872,18.168582,18.0,32.0,5.0,5.819259,13.970588,13.5,26.0,5.0,4.238963,13.709677,13.0,29.0,5.0,4.397413,15.334812,15.0,31.0,4.0,4.826879,16.506849,16.0,38.0,5.0,5.568069,12.371936,14.409526,15.092037,14.605592,12.826156,16.049444,12.772816,13.687095,12.05477,12.75611,10.928626,11.068115,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,01_01
1,11.0,2,2,2,1,0,0,18.716667,19.0,38.0,6.0,6.036869,14.4,14.0,19.0,11.0,2.966479,18.793103,19.0,50.0,6.0,5.943590,13.970588,13.5,26.0,5.0,4.238963,13.709677,13.0,29.0,5.0,4.397413,15.334812,15.0,31.0,4.0,4.826879,16.506849,16.0,38.0,5.0,

In [24]:
from sklearn.model_selection import train_test_split
isfirst = 1
for s_i in np.unique(store_item):
    one_store_item = df_train_sample[
        df_train_sample.store_item == s_i
    ]
    print(one_store_item.shape)
    y = one_store_item['sales']
    X = one_store_item.drop(['sales','store_item'],1)
    X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.9, random_state=42)    
#    X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train,test_size=0.5, random_state=42)
    if isfirst == 1:
        sample_trainX = pd.DataFrame(columns = X_train.columns)
        sample_trainY = []
#         sample_validX = pd.DataFrame(columns = X_test.columns)
#         sample_validY = []        
        sample_testX = pd.DataFrame(columns = X_test.columns)
        sample_testY = []
        isfirst = 0
    sample_trainX = pd.concat([sample_trainX,X_train])
    sample_trainY  = sample_trainY + y_train.tolist()
    
#     sample_validX = pd.concat([sample_validX,X_valid])
#     sample_validY  = sample_validY + y_valid.tolist()
    
    sample_testX = pd.concat([sample_testX,X_test])
    sample_testY  = sample_testY + y_test.tolist()
    

(1826, 197)
(1826, 197)
(1826, 197)
(1826, 197)
(1826, 197)
(1826, 197)
(1826, 197)
(1826, 197)
(1826, 197)
(1826, 197)
(1826, 197)
(1826, 197)
(1826, 197)
(1826, 197)
(1826, 197)
(1826, 197)
(1826, 197)
(1826, 197)
(1826, 197)
(1826, 197)
(1826, 197)
(1826, 197)
(1826, 197)
(1826, 197)
(1826, 197)
(1826, 197)
(1826, 197)
(1826, 197)
(1826, 197)
(1826, 197)
(1826, 197)
(1826, 197)
(1826, 197)
(1826, 197)
(1826, 197)
(1826, 197)
(1826, 197)
(1826, 197)
(1826, 197)
(1826, 197)
(1826, 197)
(1826, 197)
(1826, 197)
(1826, 197)
(1826, 197)
(1826, 197)
(1826, 197)
(1826, 197)
(1826, 197)
(1826, 197)
(1826, 197)
(1826, 197)
(1826, 197)
(1826, 197)
(1826, 197)
(1826, 197)
(1826, 197)
(1826, 197)
(1826, 197)
(1826, 197)
(1826, 197)
(1826, 197)
(1826, 197)
(1826, 197)
(1826, 197)
(1826, 197)
(1826, 197)
(1826, 197)
(1826, 197)
(1826, 197)
(1826, 197)
(1826, 197)
(1826, 197)
(1826, 197)
(1826, 197)
(1826, 197)
(1826, 197)
(1826, 197)
(1826, 197)
(1826, 197)
(1826, 197)
(1826, 197)
(1826, 197)
(182

In [26]:
print(sample_trainX.shape, len(sample_trainY))
#print(sample_validX.shape, len(sample_validY))
print(sample_testX.shape, len(sample_testY))

(91000, 195) 91000
(822000, 195) 822000


## 3. Train Tree Model

In [27]:
import numpy as np
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [28]:
def smape(y_true, y_pred):
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 200.0
    diff = np.abs(y_true - y_pred) / denominator
    diff[denominator == 0] = 0.0
    return np.nanmean(diff)

#### 1. Linear Reg

In [30]:
from sklearn.linear_model import LinearRegression
reg1 = LinearRegression()
reg1.fit(sample_trainX, sample_trainY)
sample_testY_pred = reg1.predict(sample_testX)
print('SMAPE',smape(sample_testY, sample_testY_pred))
print('MAE:',mean_absolute_error(sample_testY, sample_testY_pred))
print('MSE:',mean_squared_error(sample_testY, sample_testY_pred))
print('R2',r2_score(sample_testY, sample_testY_pred))

SMAPE 12.7075461453784
MAE: 5.54386746010699
MSE: 51.980496759067115
R2 0.9375068261569478


#### 2. Ridge Reg

In [31]:
from sklearn.linear_model import Ridge
reg2 = Ridge(alpha = 0.1)
reg2.fit(sample_trainX, sample_trainY)
sample_testY_pred = reg2.predict(sample_testX)
print('SMAPE',smape(sample_testY, sample_testY_pred))
print('MAE:',mean_absolute_error(sample_testY, sample_testY_pred))
print('MSE:',mean_squared_error(sample_testY, sample_testY_pred))
print('R2',r2_score(sample_testY, sample_testY_pred))

SMAPE 12.70293840968744
MAE: 5.5343053277547405
MSE: 51.80393186197332
R2 0.9377191000191967


#### 3. Lasso

In [27]:
from sklearn.linear_model import Lasso
reg3 = Lasso(alpha = 0.5) ###small
reg3.fit(sample_trainX, sample_trainY)
sample_testY_pred = reg3.predict(sample_testX)
print('SMAPE',smape(sample_testY, sample_testY_pred))
print('MAE:',mean_absolute_error(sample_testY, sample_testY_pred))
print('MSE:',mean_squared_error(sample_testY, sample_testY_pred))
print('R2',r2_score(sample_testY, sample_testY_pred))

SMAPE 12.433406349368493
MAE: 5.538308415598498
MSE: 52.17073412216728
R2 0.9372781146719698


#### 4. KNN

In [ ]:
# from sklearn.neighbors import KNeighborsRegressor
# reg4 = KNeighborsRegressor(30)
# reg4.fit(sample_trainX, sample_trainY)
# sample_testY_pred = reg4.predict(sample_testX)
# print('SMAPE',smape(sample_testY, sample_testY_pred))
# print('MAE:',mean_absolute_error(sample_testY, sample_testY_pred))
# print('MSE:',mean_squared_error(sample_testY, sample_testY_pred))
# print('R2',r2_score(sample_testY, sample_testY_pred))

#### 5. Decision Tree

In [71]:
from sklearn.tree import DecisionTreeRegressor
reg5 = DecisionTreeRegressor()
reg5.fit(sample_trainX, sample_trainY)
sample_testY_pred = reg5.predict(sample_testX)
print('SMAPE',smape(sample_testY, sample_testY_pred))
print('MAE:',mean_absolute_error(sample_testY, sample_testY_pred))
print('MSE:',mean_squared_error(sample_testY, sample_testY_pred))
print('R2',r2_score(sample_testY, sample_testY_pred))

SMAPE 13.069549962267502
MAE: 5.75597201946472
MSE: 73.15275060827251
R2 0.9120526380875191


#### 6. Random Forest

In [28]:
from sklearn.ensemble import RandomForestRegressor
reg6 = RandomForestRegressor(n_estimators = 250)
reg6.fit(sample_trainX, sample_trainY)
sample_testY_pred = reg6.predict(sample_testX)
print('SMAPE',smape(sample_testY, sample_testY_pred))
print('MAE:',mean_absolute_error(sample_testY, sample_testY_pred))
print('MSE:',mean_squared_error(sample_testY, sample_testY_pred))
print('R2',r2_score(sample_testY, sample_testY_pred))

SMAPE 9.713709618721637
MAE: 4.219963489051094
MSE: 34.13988338906082
R2 0.9589555737125203


#### 7. Gradient Boosting

In [29]:
from sklearn.ensemble import GradientBoostingRegressor
reg7 = GradientBoostingRegressor(n_estimators = 250)
reg7.fit(sample_trainX, sample_trainY)
sample_testY_pred = reg7.predict(sample_testX)
print('SMAPE',smape(sample_testY, sample_testY_pred))
print('MAE:',mean_absolute_error(sample_testY, sample_testY_pred))
print('MSE:',mean_squared_error(sample_testY, sample_testY_pred))
print('R2',r2_score(sample_testY, sample_testY_pred))

SMAPE 11.151147024460311
MAE: 4.9188048705283425
MSE: 40.99590805438826
R2 0.950712967966206


## 4. Cross Validation

#### 3. random forest cv

In [29]:
#train rf
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer

learner=RandomForestRegressor(random_state=1000,\
                              max_features='auto')
scorer = make_scorer(r2_score)    
# Apply  GridSearchCV to do the cross validation and find optimal parameters
parameters = {'n_estimators':[1, 10, 50, 100, 200, 300, 500]}
grid_obj3 =  GridSearchCV(learner,parameters,cv=2,\
                          scoring=scorer,return_train_score=True)
grid_obj3.fit(sample_trainX, sample_trainY)

GridSearchCV(cv=2, error_score='raise-deprecating',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
           oob_score=False, random_state=1000, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'n_estimators': [1, 10, 50, 100, 200, 300, 500]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=make_scorer(r2_score), verbose=0)

In [33]:
cv3_results = pd.DataFrame(grid_obj3.cv_results_)
cv3_results.to_csv('cv3_results.csv')
cv3_results

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_estimators,params,split0_test_score,split1_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,mean_train_score,std_train_score
0,4.818075,0.204251,0.636529,0.021700,1,{'n_estimators': 1},0.912608,0.914690,0.913649,0.001041,7,0.968882,0.968408,0.968645,0.000237
1,37.475104,0.058574,0.723888,0.001150,10,{'n_estimators': 10},0.955127,0.957082,0.956105,0.000977,6,0.992137,0.992127,0.992132,0.000005
2,195.684458,0.025723,1.545659,0.078411,50,{'n_estimators': 50},0.959255,0.960422,0.959839,0.000584,5,0.994162,0.994206,0.994184,0.000022
3,427.295588,6.672679,2.593639,0.387795,100,{'n_estimators': 100},0.959856,0.960994,0.960425,0.000569,4,0.994435,0.994478,0.994457,0.000021
4,732.978023,0.518790,3.805050,0.012787,200,{'n_estimators': 200},0.960187,0.961164,0.960676,0.000489,3,0.994557,0.994588,0.994572,0.000016
5,1101.175033,1.224910,5.407158,0.027131,300,{'n_estimators': 300},0.960289,0.961233,0.960761,0.000472,2,0.994606,0.994624,0.994615,0.000009
6,1831.444573,2.890076,8.146747,0.971455,500,{'n_estimators': 500},0.960304,0.961247,0.960775,0.000472,1,0.994637,0.994658,0.994648,0.000011


In [31]:
best_est3 = grid_obj3.best_estimator_
best_est3

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=None,
           oob_score=False, random_state=1000, verbose=0, warm_start=False)

In [32]:
best_est3.fit(sample_trainX, sample_trainY)
sample_testY_pred = best_est3.predict(sample_testX)
print('SMAPE',smape(sample_testY, sample_testY_pred))
print('MAE:',mean_absolute_error(sample_testY, sample_testY_pred))
print('MSE:',mean_squared_error(sample_testY, sample_testY_pred))
print('R2',r2_score(sample_testY, sample_testY_pred))

SMAPE 9.707165886001972
MAE: 4.216689350364963
MSE: 34.03087658706081
R2 0.9590866263467234
